# LCEL(LangChain Expression Language)復習

## 1. 基本LCEL
* 各Runnableを「|」でつなぐ
* Runnableの実行方法
    * invoke
    * stream
    * batch - 複数入力をまとめて処理
* 以下は、非同期実行用
    * ainvoke
    * astream
    * abatch

In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

# chain
chain = prompt | model | output_parser

for chunk in chain.stream({"dish": "カレー"}):
    print(chunk, end="", flush=True)

カレーのレシピをご紹介します。シンプルで美味しい基本のカレーを作りましょう。

### 材料（4人分）
- 鶏肉（もも肉または胸肉）: 400g
- 玉ねぎ: 2個
- にんじん: 1本
- じゃがいも: 2個
- カレールー: 1箱（約200g）
- サラダ油: 大さじ2
- 水: 800ml
- 塩: 適量
- 胡椒: 適量
- お好みでガーリックパウダーや生姜: 適量

### 作り方
1. **材料の下ごしらえ**:
   - 鶏肉は一口大に切り、塩と胡椒をふっておきます。
   - 玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切ります。

2. **炒める**:
   - 大きめの鍋にサラダ油を熱し、玉ねぎを中火で炒めます。玉ねぎが透明になるまで炒めます。
   - 鶏肉を加え、表面が白くなるまで炒めます。

3. **野菜を加える**:
   - にんじんとじゃがいもを鍋に加え、全体をよく混ぜます。

4. **煮る**:
   - 水を加え、強火で煮立たせます。煮立ったら、アクを取り除き、蓋をして中火にし、約15分煮ます。

5. **カレールーを加える**:
   - 火を止めてカレールーを加え、よく溶かします。再び弱火にし、10分ほど煮込みます。お好みでガーリックパウダーや生姜を加えて風味を調整します。

6. **味を調える**:
   - 最後に塩で味を調整し、全体がなじむまでさらに5分ほど煮ます。

7. **盛り付け**:
   - ご飯と一緒に盛り付けて、お好みで福神漬けやらっきょうを添えて完成です。

### おすすめのトッピング
- 煮卵
- チーズ
- 青ねぎやパセリの刻んだもの

この基本のカレーはアレンジがしやすいので、野菜や肉を変えて自分好みのカレーを楽しんでください！

## 2. Zero-shot CoTでステップバイステップで考えさせるChainと結論を作成するChainの連結

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [2]:
### Zero-shot CoTでステップバイステップで考えさせるChain
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

cot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーの質問にステップバイステップで回答してください。"),
        ("human", "{question}"),
    ]
)

cot_chain = cot_prompt | model | output_parser


### 上記回答から結論を抽出するChain
summarize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ステップバイステップで考えた回答から結論だけ抽出してください。"),
        ("human", "{text}"),
    ]
)

summarize_chain = summarize_prompt | model | output_parser


### ２つのchainを連結
cot_summarize_chain = cot_chain | summarize_chain
cot_summarize_chain.invoke({"question": "10+2*3"})

'結論: \\(10 + 2 * 3 = 16\\) です。'

## 3. RunnableLambda - 任意の関数のRunnable化
* LLMの生成したテキストに対して、小文字を大文字に変換する処理をChain化

### 3-1

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda

In [5]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "{input}",)
    ]
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
output_parser = StrOutputParser()

# 小文字を大文字に変換する関数
def upper(text: str) -> str:
    return text.upper()

chain = prompt | model | output_parser | RunnableLambda(upper)

output = chain.invoke({"input": "Hello!"})
print(output)



HELLO! HOW CAN I ASSIST YOU TODAY?


### 3-2. chainデコレータ(@chain)を使った実装

In [6]:
from langchain_core.runnables import chain

@chain
def upper(text: str) -> str:
    return text.upper()

chain = prompt | model | output_parser | upper

output = chain.invoke({"input": "Hello!"})
print(output)

HELLO! HOW CAN I ASSIST YOU TODAY?


### 3-3. RunnableLambdaへの自動変換
* 明示的にRunnableLambdaを作成しなくても、Runnableと任意の関数を「|」で接続できる。

In [7]:
def upper(text: str) -> str:
    return text.upper()

chain = prompt | model | output_parser | upper

output = chain.invoke({"input": "Hello!"})
print(output)

HELLO! HOW CAN I ASSIST YOU TODAY?


#### Runnableの入力型と出力型に注意すること
```python
def upper(text: str) -> str:
    return text.upper()

chain = prompt | model | upper

output = chain.invoke({"input": "Hello!"})
```

上記コードでは、以下のエラーが発生する。
```bash
AttributeError: 'AIMessage' object has no attribute 'upper'
```

原因として、modelがAIMessageを出力するのに対して、自作upper関数は入力としてstrを期待しているため。
そのため、前のRunnerの出力と次のRunnerの入力の型は一致していないといけない。

### 自作関数のストリーミング処理について
* ジェネレータ関数を用いることで、自作関数をストリーミング対応にできる

## 4. RunnableParallelによる複数Runnerの並列化

### 4-1. 一つのinputに対して2つのLLMの実行

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import pprint
from langchain_core.runnables import RunnableParallel

In [13]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
output_parser = StrOutputParser()

# 楽観的な意見を言うLLM
optimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは楽観主義者です。ユーザーの入力に対して楽観的な意見を下さい。"),
        ("human", "{topic}"),
    ]
)
optimistic_chain = optimistic_prompt | model | output_parser

# 楽観的な意見を言うLLM
pessimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは悲観主義者です。ユーザーの入力に対して悲観的な意見を下さい。"),
        ("human", "{topic}"),
    ]
)
pessimistic_chain = pessimistic_prompt | model | output_parser

# 並列実行
parallel_chain = RunnableParallel(
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
    }
)

output = parallel_chain.invoke({"topic": "生成AIの進化について"})
pprint.pprint(output)

{'optimistic_opinion': '生成AIの進化は本当に素晴らしいですね！技術が進むことで、私たちの生活がより便利で豊かになる可能性が広がっています。クリエイティブな作業や問題解決の手助けをしてくれるAIが増えてきて、私たちのアイデアを実現するためのパートナーとして活躍しています。これからも新しい発見や革新が続くことで、私たちの未来はますます明るくなるでしょう！どんな素晴らしいことが待っているのか、ワクワクしますね！',
 'pessimistic_opinion': '生成AIの進化は確かに目覚ましいものですが、その裏には多くの懸念が潜んでいます。技術が進化することで、私たちの仕事が奪われたり、情報の信頼性が低下したりするリスクが高まっています。さらに、AIが生成するコンテンツが人間の創造性を脅かし、私たちの思考や感情に悪影響を及ぼす可能性もあります。結局のところ、便利さの裏には常に危険が潜んでいるのです。私たちがこの技術をどれだけ賢く使おうとも、その影響を完全にコントロールすることは難しいでしょう。'}


### 4-2. RunnableParallelの出力をRunnableの入力に連結する
* 楽観的な意見と悲観的な意見を出したうえで、客観的にまとめるChain

In [15]:
synthesize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは客観的なAIです。２つの意見をまとめてください。"),
        ("human", "楽観的意見：{optimistic_opinion}\n悲観的意見：{pessimistic_opinion}"),
    ]
)

synthesize_chain = (
    RunnableParallel(
        {
            "optimistic_opinion": optimistic_chain,
            "pessimistic_opinion": pessimistic_chain,
        }
    )
    | synthesize_prompt
    | model
    | output_parser
)

output = synthesize_chain.invoke({"topic": "生成AIの進化について"})
print(output)

生成AIの進化については、楽観的な意見と悲観的な意見が存在します。楽観的な見方では、生成AIの技術が進むことで私たちの生活が便利で豊かになり、クリエイティブな作業や問題解決のパートナーとしての役割を果たすことが期待されています。この進化により、新しい発見や革新が続き、未来が明るくなる可能性があるとされています。

一方で、悲観的な見方では、生成AIの進化には多くの懸念が伴います。仕事の喪失や情報の信頼性の低下、さらにはオリジナリティやクリエイティビティの喪失といったリスクが指摘されています。便利さの裏には問題が潜んでおり、この技術をどのように扱うかが重要であるとされています。

総じて、生成AIの進化は多くの可能性を秘めている一方で、慎重な対応が求められる複雑な状況であると言えるでしょう。


### 4-3. RunnableParallelへの自動変換
* キーがstrで値がRunnable（または、Runnableに自動変換できる関数など）であるdictは、RunnableParallelに自動変換される。）

In [ ]:
synthesize_chain = (
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
    } # 上記は、RunnableParallelに自動変換される
    | synthesize_prompt
    | model
    | output_parser
)

### 4-4. itemgetterを使う
* `itemgetter`を使うと、dictなどから値を取り出す関数を簡単に作れる.
以下は、『{"topic": "生成AIの進化について"}』というdictから、itemgetter("topic")を使ってtopicを取り出す例

In [17]:
from operator import itemgetter

topic_getter = itemgetter("topic")
topic = topic_getter({"topic": "生成AIの進化について"})
print(topic)

生成AIの進化について


以下では、『{"topic": "生成AIの進化について"}』から`itemgetter("topic")`で値を取り出し、`ChatPromptTemplate`の`{topic}`の箇所に穴埋めしている例

In [18]:
synthesize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは客観的なAIです。{topic}について２つの意見をまとめてください。"),
        ("human", "楽観的意見：{optimistic_opinion}\n悲観的意見：{pessimistic_opinion}"),
    ]
)

synthesize_chain = (
    {"optimistic_opinion": optimistic_chain,
     "pessimistic_opinion": pessimistic_chain,
     "topic": itemgetter("topic") # RunnableLambda(itemgetter("topic"))に自動変換される
     }
     | synthesize_prompt
     | model
     | output_parser
)

output = synthesize_chain.invoke({"topic": "生成AIの進化について"})
print(output)

**楽観的意見**：生成AIの進化は、私たちの生活をより便利で豊かにする可能性を秘めています。技術の進歩により、クリエイティブな作業や問題解決の支援を行うAIが登場し、私たちのアイデアや想像力を引き出す手助けをしてくれるでしょう。未来において、生成AIが社会に与える影響や新しい発見、革新に対する期待が高まります。

**悲観的意見**：生成AIの進化には多くの懸念が伴います。技術の進化が進むことで、仕事の喪失や情報の信頼性の低下といったリスクが増大します。また、AIが生成するコンテンツが人間の創造性を脅かし、思考や感情に悪影響を及ぼす可能性もあります。便利さの裏には常に危険が潜んでいるため、慎重な対応が求められます。


## 5. RunnablePassthrough - 入力をそのまま出力する
* RunnableParallelで、その要素の一部で入力の値をそのまま出力したい場合などに利用

In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.retrievers import TavilySearchAPIRetriever
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv
load_dotenv()

True

In [24]:
prompt = ChatPromptTemplate.from_template(
    """
    以下の文脈だけを踏まえて質問に答えてください。
                                          
    文脈：'''
    {context}
    '''

    質問：{question}
    """
    )

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# webデータベクトル検索器
retriever = TavilySearchAPIRetriever(k=3)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

output = chain.invoke("東京の今日の天気は？")
print(output)

東京の今日の天気は曇のち雨です。昼頃から所々で雨雲が湧き、夜は広く雨になる予報です。


* RunnablePassthroughは、入力値をそのまま出力する。

### 5-2. assign - RunnableParallelの出力に値を追加
* 上記では、LLMが生成した最終的な回答だけがChain全体の出力になっている。しかし、retreiverの検索結果もChain全体の出力に含めたい時がある。その時に使えるメソッド

In [25]:
import pprint

chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | RunnablePassthrough.assign(answer=prompt | model | StrOutputParser())

output = chain.invoke("東京の今日の天気は？")
pprint.pprint(output)

{'answer': '東京の今日の天気は「曇のち雨」です。昼頃から所々で雨雲が湧き、夜には広く雨が降る予報です。',
 'context': [Document(metadata={'title': '東京都の天気 - 日本気象協会 tenki.jp', 'source': 'https://tenki.jp/forecast/3/16/', 'score': 0.7803451, 'images': []}, page_content='# tenki.jp 雨雲レーダー) 天気図 PM2.5分布予測 地震情報 日直予報士 熱中症情報 東京都の天気 ### 05月29日(木) 東京都の天気 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 晴時々曇 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 最新の天気履歴 渋谷区 曇のち雨 豊島区 曇のち雨 江東区 曇のち雨 港区 曇のち雨 大田区 曇のち雨 府中市 曇のち雨 奥多摩町 曇のち雨 関東・甲信 ### 気象予報士のポイント解説(日直予報士) newマーク 関東甲信\u3000今日29日は天気下り坂\u3000昼頃から所々で雨雲が湧く\u3000夜は広く雨に 関東甲信\u3000今日29日は天気下り坂\u3000昼頃から所々で雨雲が湧く\u3000夜は広く雨に newマーク 今日29日は気圧が低下\u3000沖縄や九州から近畿は影響度「大」の所も\u3000頭痛など注意 今日29日は気圧が低下\u3000沖縄や九州から近畿は影響度「大」の所も\u3000頭痛など注意 今日29日\u3000九州から関東は次第に雨\u3000東北と北海道は晴れて気温上昇\u3000真夏日も 今日29日\u3000九州から関東は次第に雨\u3000東北と北海道は晴れて気温上昇\u3000真夏日も ### 東京都各地の天気 #### 東京23区 #### 多摩 #### 伊豆諸島北部(大島) #### 伊豆諸島南部(八丈島) #### 小笠原諸島(父島) ### おすすめ記事 LINEの友達追加 ### 天気ガイド 雨雲 ### 注目の情報 アプリに便利なサブスクプラン開始 「tenki.jpライト」なら現在地の雨雲接近通知が受け取れる！ 新サービス「気圧予報」 気圧変化を確認して、頭痛やめまい、倦怠感

または、以下の方法がある：

In [ ]:
chain = RunnableParallel(
    {
        "question": RunnablePassthrough(),
        "context": retriever,
    }
).assign(answer=prompt | model | StrOutputParser())

中間値を出力する方法として、ほかに`astream_events`を使う方法もある。

# RunnablePassthroughのassingメソッドについて

以下では、LangChain Expression Language（LCEL）における `assign` メソッドの役割と使い方、そしてコード内で渡される引数がどのように作用するのかを解説します。

## 概要

`RunnablePassthrough.assign()` は、チェーンの途中で「入力状態の辞書」をそのまま引き継ぎつつ、新たなキーと値（ランナブル）を追加し、次のステップへの入力として渡すためのヘルパーです。これにより、複数の並行処理や段階的なデータ追加が容易になります。たとえば、Retriever で取り出した文脈（`context`）やプロンプト結果をまとめて次に渡す際に多用されます。

---

## 1. `assign` の役割と動作原理

* **元の状態を保持**しつつ、新しいフィールドを「付け足す」
* 返り値は常に「更新後の全辞書」
* 内部的には、入力の辞書オブジェクトをクローンし、新たに指定したキーでランナブル（関数やチェーン）を実行した結果を追加します。
* LCEL パイプでは、複数の情報ストリームをまとめるときに特に便利です。

---

## 2. `assign` メソッドのシグネチャ

```python
RunnablePassthrough.assign(**kwargs)
```

* `**kwargs` に渡すのは、

  * **キー名**：出力の辞書に追加されるフィールド名
  * **値**：`Runnable`（たとえば、`prompt | model | parser` など）もしくは関数オブジェクト

たとえば：

```python
RunnablePassthrough.assign(
  answer=prompt | model | StrOutputParser()
)
```

これは、

1. 入力辞書をそのまま受け取り
2. `prompt | model | StrOutputParser()` を実行
3. その結果を `"answer"` キーにセット
4. 更新済み辞書を返す
   という動作を一行で実現します。

---

## 3. コード全体のフロー解説

```python
import pprint
prompt = ChatPromptTemplate.from_template(
    "以下文脈を踏まえて質問に答えてください。　文脈：'{context}'　質問：{question}"
)
model = ChatOpenAI(...)
retriever = TavilySearchAPIRetriever(k=3)

chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | RunnablePassthrough.assign(
    answer=prompt | model | StrOutputParser()
)
```

1. **Parallel 要素の定義**

   ```python
   {
     "question": RunnablePassthrough(),
     "context": retriever,
   }
   ```

   * ここで、入力辞書から `"question"` キーはそのまま・`"context"` キーは Retriever 結果として並行取得される状態を作成します。

2. **`assign(answer=…)` の適用**

   * `RunnableParallel` の出力である辞書（例：`{"question": "...", "context": [...]}`）を受け取り
   * `prompt | model | StrOutputParser()` をその辞書をもとに実行
   * 生成された文字列を `"answer"` キーに追加
   * 最終的に `{ "question": "...", "context": [...], "answer": "モデル出力" }` を返します。

---

## 4. LCEL におけるデータ受け渡しパターン

* **`RunnableParallel`**：複数のランナブルを同じ入力に並行適用し、キーごとの出力をまとめる。
* **`RunnablePassthrough()`**：特定のキーを元のまま次ステップに通過させる“パススルー”用。
* **`assign`**：上記でまとめた辞書に対し、追加のランナブル実行結果を付与する。

これらを組み合わせることで、以下のような典型的フローを実装できます：

```python
# 1) Retriever で複数ソースから context_a, context_b を取得
retrieval = RunnableParallel(
    context_a=retriever_a, context_b=retriever_b,
    question=RunnablePassthrough()
)

# 2) assign で answer キーを追加
chain = retrieval | RunnablePassthrough.assign(
    answer=prompt | model | parser
)
```

---

## 5. まとめ

* `assign` は **“元の辞書を保持しつつ新しいフィールドを付与する”** LCEL の基本プリミティブです。
* キー名とランナブル（関数・チェーン）を渡すだけで、その実行結果を辞書にマージし、後続ステップへ繋げます。
* `RunnableParallel`＋`RunnablePassthrough.assign` を組み合わせることで、マルチソースからの文脈取得 → プロンプト生成 → モデル呼び出し → 結果格納…といった複雑なフローも簡潔に記述できます。

---

#### 参考

1. How to add values to a chain’s state (assign の基礎) ([python.langchain.com][1])
2. RunnablePassthrough API リファレンス ([python.langchain.com][2])
3. データを次ステップへそのまま通す方法 ([python.langchain.com][3])
4. LCEL フロー例（Pinecone 解説） ([pinecone.io][4])
5. ソースドキュメントを返す例（StackOverflow） ([stackoverflow.com][5])
6. LCEL チートシート ([python.langchain.com][6])
7. GitHub discussion: assign の活用例 ([github.com][7])
8. LCEL 変数のアクセス（StackOverflow） ([stackoverflow.com][8])
9. JavaScript版 assign 解説 ([js.langchain.com][9])
10. assign でパススルー＋追加フィールド ([github.com][10])

以上を参考に、`assign` を使いこなし、LCEL でのより複雑なチェーン構築にお役立てください。

[1]: https://python.langchain.com/docs/how_to/assign/?utm_source=chatgpt.com "How to add values to a chain's state - Python LangChain"
[2]: https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html?utm_source=chatgpt.com "RunnablePassthrough — LangChain documentation"
[3]: https://python.langchain.com/docs/how_to/passthrough/?utm_source=chatgpt.com "How to pass through arguments from one step to the next"
[4]: https://www.pinecone.io/learn/series/langchain/langchain-expression-language/?utm_source=chatgpt.com "LangChain Expression Language Explained - Pinecone"
[5]: https://stackoverflow.com/questions/77759685/how-to-return-source-documents-when-using-langchain-expression-language-lcel?utm_source=chatgpt.com "How to return source documents when using LangChain Expression ..."
[6]: https://python.langchain.com/docs/how_to/lcel_cheatsheet/?utm_source=chatgpt.com "LangChain Expression Language Cheatsheet"
[7]: https://github.com/langchain-ai/langchain/discussions/23532?utm_source=chatgpt.com "RunnablePassthrough.assign(context=(lambda x: format_docs(x ..."
[8]: https://stackoverflow.com/questions/78379953/accessing-langchain-lcel-variables-from-prior-steps-in-the-chain?utm_source=chatgpt.com "Accessing LangChain LCEL variables from prior steps in the chain"
[9]: https://js.langchain.com/docs/how_to/assign/?utm_source=chatgpt.com "How to add values to a chain's state - LangChain.js"
[10]: https://github.com/langchain-ai/langchain/discussions/18311?utm_source=chatgpt.com "How to pass arguments to a function in LCEL #18311 - GitHub"



# chain.invoke()に渡す引数の型について

`chain.invoke()` が受け取る引数は常に **ひとつだけ** です。これは LCEL（LangChain Expression Language）の設計上の制約で、複数の引数を個別に渡すことはできません。では、その「ひとつ」の引数に何を渡せばよいか、以下のポイントで整理します。

---

## 1. 何を渡すかはチェーンの最初のステップ次第

* **RunnableParallel（もしくは辞書リテラル）** でチェーンを構築した場合

  ```python
  chain = RunnableParallel({
      "question": RunnablePassthrough(),
      "context": retriever,
  }) | RunnablePassthrough.assign(
      answer = prompt | model
  )
  ```

  のように定義すると、`RunnableParallel` は全てのサブランナブルに同じ入力を同時に投げます。

  * `RunnablePassthrough()` は入力をそのまま返すランナブルなので、`"question"` キーにはそのまま渡した値が入ります。
  * `retriever` は文字列クエリを受け取る設計のランナブルなので、同じく文字列を受け取ります。

  その結果、**文字列（`str`）をひとつ**渡せばよい、ということになります。その文字列が `"question"` に入り、同じ文字列が `retriever` のクエリとして使われます。

  ```python
  result = chain.invoke("LangChainの概要を教えてください。")
  ```

  ([python.langchain.com][1])

---

## 2. dict を渡すパターン

もし最初のステップで、サブランナブルのうちどれかが **辞書型の入力**を期待している場合（たとえば複数の異なるキーを別々に扱いたいときなど）、引数には **キー名を合わせた辞書** を渡します。

```python
chain = RunnableParallel({
    "query": lambda x: x["text"],  # dict の "text" キーを参照する
    "meta": SomeRunnable(),
})
```

このようなチェーンなら、

```python
# {"text": "..."} という dict を渡す
chain.invoke({"text": "質問内容をここに", "other": "..."})
```

のように、辞書を渡す必要があります。
([python.langchain.com][2])

---

## 3. チェーンが期待する入力キーを調べる方法

実際にどんな引数を入れればいいか迷ったら、**`.input_keys`** プロパティで確認できます。

```python
print(chain.input_keys)
# → ['question']  のように出れば、文字列を入れたときに "question" として扱われます。
```

* `['question']` なら文字列を渡すと `"question"` キーに自動マッピングされる
* `['text', 'other']` など複数キーなら、それらを含む辞書を渡す

---

## 4. まとめ

1. **文字列を渡す**

   * チェーンの最初が `RunnablePassthrough()` と文字列クエリ対応の Retriever であれば、`chain.invoke("...")` で OK。
2. **辞書を渡す**

   * 最初のランナブルが辞書型入力を期待している場合は、キー名を揃えた辞書を `chain.invoke({...})` で渡す。
3. **確認は `.input_keys`**

   * `chain.input_keys` で何を要求されているかを見るのが最も確実です。

このように、**チェーンの最初にあるランナブルたちが受け取る型**を揃えれば、`chain.invoke()` の引数は文字列でも辞書でも問題なく動作します。

[1]: https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.RunnableParallel.html?utm_source=chatgpt.com "RunnableParallel — LangChain documentation"
[2]: https://python.langchain.com/docs/concepts/lcel/?utm_source=chatgpt.com "LangChain Expression Language (LCEL)"
